## Web Scraping

Importamos todas las librerías que nos pueden ser necesarias

In [2]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import scrapy
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re
import pandas as pd
import numpy as np

Seleccionamos página web. Meetup
Queremos saber los eventos de tecnología en madrid. 

In [3]:
url = 'https://www.meetup.com/es-ES/find/tech/'

In [14]:
# Exportamos la informacion con beautiful soup
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")


Queremos saber los 50 grupos relacionados con tecnología, el número de miembros y cómo se identifican a sus miembros 

In [50]:
#Realizamos la petición a la web
b = []
for item in soup.find_all('ul',{'class':'j-groupCard-list'}):
    title = [x.text.strip() for x in item.find_all('h3', {'class':'padding-none'},)]
    members = [e.text.strip() for e in item.find_all('p', {'class':'small'},)]
    b = pd.DataFrame(list(zip(title, members)))
    b.columns=['Group_name', 'Members']
    #Dividimos la columna Members entre el número de participantes y el nombre de los participantes 
    b['Numbers_of_members'], b['Name_members'] = b['Members'].str.split(' ', 1).str
    b.drop(['Members'], axis=1, inplace=True)
display(b)

,Group_name,Numbers_of_members,Name_members
0,Kaggle Days Meetup Madrid,330,Miembros
1,Madriagil - Grupo Meetup de Agilismo de Madrid,4.249,agilistas
2,Python Madrid,5.163,Pythonistas
3,WordPress Madrid,2.812,Miembros
4,KeepCoding,2.728,KeepCoders
5,Meetup BABEL,1.059,BABELIANS
6,Ironhack Madrid,5.739,Ironhackers
7,Machine Learning Spain,3.842,Machine Learners
8,Digital&Beers Madrid,1.815,Miembros
9,NWC10Lab: Transformación digital y blockchain ...,1.227,Pioneros


Ahora queremos conocer los próximos eventos en tecnología, el grupo que lo organiza y la cantidad de miembros que van a asistir. 

In [8]:
url2 = 'https://www.meetup.com/es-ES/find/events/tech/?allMeetups=false&radius=31&userFreeform=Madrid%2C+Espa%C3%B1a&mcId=z1010808&mcName=Madrid%2C+ES&eventFilter=all'

In [11]:
# Exportamos la informacion con beautiful soup
html2 = requests.get(url2).content
soup2 = BeautifulSoup(html2, "lxml")

In [26]:
#Realizamos la petición a la web
c = []
for item in soup2.find_all('div',{'class':'noNav '}, limit=9): 
    group_name = [x.text.strip() for x in item.find_all('div',{'class':'text--labelSecondary'}, limit=9)]
    event_name = [i.text.strip().split(',') for i in item.select('a..resetLink.omnCamp.omngj_sj7eall.omnrv_fe1all')]
    audience = [j.text.strip().replace('\n', ' ') for j in item.find_all('div', {'class': 'attendee-count'})]
    
c = pd.DataFrame(list(zip(group_name, event_name, audience)))
c.columns = ['Group_name', 'Event_name', 'Audience2']

#Dividimos la columna Audience2 entre el número de participantes y el nombre de los participantes 
c['Audience'], c['Name_audience'] = c['Audience2'].str.split(' ', 1).str
c.drop(['Audience2'], axis=1, inplace=True)


display (c)

,Group_name,Event_name,Audience,Name_audience
0,KeepCoding,[¿Qué es eso de programar? Programación para d...,9,KeepCoders participarán
1,Meetup de Big Data en DataHack Madrid,[¡Ven a conocer datahack Madrid!],14,Amantes del Big Data participarán
2,Conversion Rate Optimization Heroes,[[Webinar] Cómo implementar trucos psicológico...,43,CRO Heroes participarán
3,Apache Mesos Madrid,[Meetup #2: Discover how to deploy a secure Bi...,69,Mesostianos participarán
4,Codemotion Meetup en España,[WEBINAR: Crea una aplicación web con F# y Fab...,5,Members participarán
5,Madrid GUG,[Gradle Plugins en Groovy: escribir y publicar...,31,grooviers participarán
6,NEOLAND Madrid,[Fundamentos y uso de la tipografía para web/app],8,Neolandeses participarán ¡Quedan 8 plazas!
7,Icofunding Meetups: Security Tokens e ICOs en ...,[Security Tokens en España: ¿Cómo financiar tu...,15,Miembros participarán
8,SciDataUCM,[Hackeando la salud y la eterna discusión entr...,8,Miembros participarán


Nos hemos dado cuenta de que el "Event_name" en la posición 9 en la página web tiene una clase diferente al resto, por lo que nuestro web scraping a partir del elemento 9 nos da como resultado el "Group_name"[9] con el "Event_name"[10]  del siguiente evento. 

## Merge

In [48]:
#Unimos los dos Dataframe para conocer la relación entre los miembros del grupo y los asistentes a los eventos. 
Meetup = pd.merge(b, c, on='Group_name')
Meetup.drop(['Name_audience'], axis=1, inplace=True)
#Modificamos el tipo de dato de las columnas numéricas. 
Meetup[['Audience', 'Numbers_of_members']] = Meetup[['Audience', 'Numbers_of_members']].apply(pd.to_numeric)
Meetup.info()
#Hemos intentado dividir el número de participantes(Audience) entre el número de miembros pero al cambio de tipo de datos nos ha dado "Numbers of members" como floar y no int. (no podemos hacer la división. )
#Meetup['percent'] = (Meetup['Audience'] / Meetup['Numbers_of_members'])
Meetup

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 6
Data columns (total 5 columns):
Group_name            7 non-null object
Numbers_of_members    7 non-null float64
Name_members          7 non-null object
Event_name            7 non-null object
Audience              7 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 336.0+ bytes


,Group_name,Numbers_of_members,Name_members,Event_name,Audience
0,KeepCoding,2.728,KeepCoders,[¿Qué es eso de programar? Programación para d...,9
1,Codemotion Meetup en España,259.000,Members,[WEBINAR: Crea una aplicación web con F# y Fab...,5
2,Apache Mesos Madrid,348.000,Mesostianos,[Meetup #2: Discover how to deploy a secure Bi...,69
3,NEOLAND Madrid,1.413,Neolandeses,[Fundamentos y uso de la tipografía para web/app],8
4,Conversion Rate Optimization Heroes,88.000,CRO Heroes,[[Webinar] Cómo implementar trucos psicológico...,43
5,Madrid GUG,812.000,grooviers,[Gradle Plugins en Groovy: escribir y publicar...,31
6,SciDataUCM,126.000,Miembros,[Hackeando la salud y la eterna discusión entr...,8


Tenemos los 7 próximos eventos en Madrid, de tecnología con el número de miembros del grupo y la audiencia del evento. 